# Практическое задание к уроку 4

In [88]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import pandas as pd



from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
import tensorflow_hub as hub
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [106]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.1.0
Eager mode:  True
Hub version:  0.10.0
GPU is available


In [26]:
#url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

#dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
#                                    untar=True, cache_dir='.',
#                                    cache_subdir='')

#dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

#train_dir = os.path.join(dataset_dir, 'train')
#os.listdir(train_dir)

['labeledBow.feat',
 'neg',
 'pos',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

In [27]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [28]:
import glob
reviews_train = []
labels_train=[]
for file_name in glob.glob("aclImdb/train/pos/*.txt"):
    with open(file_name, 'r', encoding='utf8') as f:
        reviews_train.append([f.read().strip(),1])
        labels_train.append(1)
for file_name in glob.glob("aclImdb/train/neg/*.txt"):
    with open(file_name, 'r', encoding='utf8') as f:
        reviews_train.append([f.read().strip(),0])
        labels_train.append(0)

labels_test=[]
reviews_test = []
for file_name in glob.glob("aclImdb/test/pos/*.txt"):
    with open(file_name, 'r', encoding='utf8') as f:
        reviews_test.append([f.read().strip(),1])
        labels_test.append(1)
for file_name in glob.glob("aclImdb/test/neg/*.txt"):
    with open(file_name, 'r', encoding='utf8') as f:
        reviews_test.append([f.read().strip(),0])
        labels_test.append(0)

In [29]:
train=pd.DataFrame(reviews_train,columns=['text','label'])
test=pd.DataFrame(reviews_test,columns=['text','label'])

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm

CountVectorizer 

In [31]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(train['text'],train['label'])
xtrain_count =  count_vect.transform(train['text'])
xtest_count =  count_vect.transform(test['text'])

In [32]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid,valid_y, is_neural_net=False):
    classifier.fit(feature_vector_train, label)
    
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [33]:
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train['label'], xtest_count,test['label'])
print("LR, Count Vectors: ", accuracy)

LR, Count Vectors:  0.86836


TfidfVectorizer

In [34]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(train['text'],train['label'])
xtrain_tfidf =  tfidf_vect.transform(train['text'])
xvalid_tfidf =  tfidf_vect.transform(test['text'])

In [35]:
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train['label'], xvalid_tfidf,test['label'])
print("LR, Tfid Vectors: ", accuracy)

LR, Tfid Vectors:  0.8854


2. Обучить вашу архитектуре сети возможно туже что и была на занятии проверить
•	взять предобученный эмбединг(к примеру word2vec) и загрузить в слой Embedding
•	взять слой Embedding без предобученных весов
Сравнить все подходы в том числе и полносвязанную сеть что лучше отработало


In [36]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [51]:
max_features = 10000
sequence_length = 250
embedding_dim = 16

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [52]:
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim,),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(145),
  layers.Dense(1)])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160016    
_________________________________________________________________
dropout (Dropout)            (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 145)               2465      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 146       
Total params: 162,627
Trainable params: 162,627
Non-trainable params: 0
________________________________________________

In [62]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['binary_accuracy']) 

In [94]:
batch_size=32
raw_train = tf.data.Dataset.from_tensor_slices((train['text'],train['label']))
raw_test = tf.data.Dataset.from_tensor_slices((test['text'],test['label']))

raw_train_ds = raw_train.batch(batch_size)
raw_test_ds=raw_test.batch(batch_size)

In [95]:
train_ds = raw_train_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [96]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [66]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=epochs)

Train for 782 steps, validate for 782 steps
Epoch 1/10
782/782 [==============================] - 15s 20ms/step - loss: 0.5423 - binary_accuracy: 0.7746 - val_loss: 2.3691 - val_binary_accuracy: 0.5000
Epoch 2/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3583 - binary_accuracy: 0.8643 - val_loss: 2.2501 - val_binary_accuracy: 0.5000
Epoch 3/10
782/782 [==============================] - 12s 16ms/step - loss: 0.3844 - binary_accuracy: 0.8434 - val_loss: 2.3562 - val_binary_accuracy: 0.5000
Epoch 4/10
782/782 [==============================] - 12s 15ms/step - loss: 0.3758 - binary_accuracy: 0.8455 - val_loss: 2.4072 - val_binary_accuracy: 0.5000
Epoch 5/10
782/782 [==============================] - 13s 17ms/step - loss: 0.3855 - binary_accuracy: 0.8459 - val_loss: 2.1964 - val_binary_accuracy: 0.5000
Epoch 6/10
782/782 [==============================] - 12s 15ms/step - loss: 0.3924 - binary_accuracy: 0.8330 - val_loss: 2.2465 - val_binary_accuracy: 0.5000
Epoch 7/

In [68]:
loss, accuracy = model.evaluate(train_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

782/782 [==============================] - 4s 5ms/step - loss: 2.4870 - binary_accuracy: 0.5000
Loss:  2.4869581636117624
Accuracy:  0.5


 взять предобученный эмбединг(к примеру word2vec) и загрузить в слой Embedding 

In [89]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
embedding_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)


In [98]:
model = tf.keras.Sequential()
model.add(embedding_layer)
model.add(tf.keras.layers.Dense(145, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_6 (Dense)              (None, 145)               3045      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 146       
Total params: 403,211
Trainable params: 403,211
Non-trainable params: 0
_________________________________________________________________


In [101]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [104]:
epochs = 10
history = model.fit(
    raw_train_ds,
    validation_data=raw_test_ds,
    epochs=epochs)

Train for 782 steps, validate for 782 steps
Epoch 1/10
782/782 [==============================] - 18s 24ms/step - loss: 0.1954 - accuracy: 0.9717 - val_loss: 6.3246 - val_accuracy: 0.5002
Epoch 2/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2908 - accuracy: 0.9458 - val_loss: 3.8305 - val_accuracy: 0.5020
Epoch 3/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2077 - accuracy: 0.9469 - val_loss: 2.8921 - val_accuracy: 0.5016
Epoch 4/10
782/782 [==============================] - 18s 23ms/step - loss: 0.1740 - accuracy: 0.9480 - val_loss: 2.8517 - val_accuracy: 0.5027
Epoch 5/10
782/782 [==============================] - 18s 23ms/step - loss: 0.1500 - accuracy: 0.9541 - val_loss: 2.4507 - val_accuracy: 0.5048
Epoch 6/10
782/782 [==============================] - 18s 23ms/step - loss: 0.1292 - accuracy: 0.9582 - val_loss: 2.4041 - val_accuracy: 0.5068
Epoch 7/10
782/782 [==============================] - 18s 23ms/step - loss: 0.1125 - accurac

In [107]:
loss, accuracy = model.evaluate(raw_test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

782/782 [==============================] - 8s 10ms/step - loss: 2.2536 - accuracy: 0.5428
Loss:  2.253603813271826
Accuracy:  0.5428


Часть 2 дополню чуть познее 